## 135. Candy
- Description:
  <blockquote>
    There are `n` children standing in a line. Each child is assigned a rating value given in the integer array `ratings`.
     
    You are giving candies to these children subjected to the following requirements:
     
    - Each child must have at least one candy.
    - Children with a higher rating get more candies than their neighbors.
     
    Return  *the minimum number of candies you need to have to distribute the candies to the children* .
     
    **Example 1:**
    **Input:** ratings = [1,0,2]
    **Output:** 5
    **Explanation:** You can allocate to the first, second and third child with 2, 1, 2 candies respectively.
     
    **Example 2:**
    **Input:** ratings = [1,2,2]
    **Output:** 4
    **Explanation:** You can allocate to the first, second and third child with 1, 2, 1 candies respectively.
    The third child gets 1 candy because it satisfies the above two conditions.
     
    **Constraints:**
     
    - `n == ratings.length`
    - `1 <= n <= 2 * 104`
    - `0 <= ratings[i] <= 2 * 104`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/candy/description)

- Topics: Array, Greedy, Two Pass

- Difficulty: Hard

- Resources: example_resource_URL

### Solution 1, TLE, Brute Forece
Solution description
- Time Complexity: O(N^2)
  - The while loop may run up to O(n) iterations
  - Each iteration scans all n elements → O(n) per iteration
  - Total: O(n) × O(n) = O(n²)
- Space Complexity: O(N)
  - One array, candies array of size n is used.

In [ ]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        candies = [1] * len(ratings)
        hasChanged = True
        while hasChanged:
            hasChanged = False
            for i in range(len(ratings)):
                if (
                    i != len(ratings) - 1
                    and ratings[i] > ratings[i + 1]
                    and candies[i] <= candies[i + 1]
                ):
                    candies[i] = candies[i + 1] + 1
                    hasChanged = True
                if (
                    i > 0
                    and ratings[i] > ratings[i - 1]
                    and candies[i] <= candies[i - 1]
                ):
                    candies[i] = candies[i - 1] + 1
                    hasChanged = True
        return sum(candies)

### Solution 2, Two pass, Two array approach for left to right and right to left processing of candies
Solution description
- Time Complexity: O(N)
  - left2right and right2left arrays are traversed thrice.
- Space Complexity: O(N)
  -  Two arrays left2right and right2left of size n are used.

In [ ]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        sum = 0
        n = len(ratings)
        left2right = [1] * n
        right2left = [1] * n

        for i in range(1, n):
            if ratings[i] > ratings[i - 1]:
                left2right[i] = left2right[i - 1] + 1

        for i in range(n - 2, -1, -1):
            if ratings[i] > ratings[i + 1]:
                right2left[i] = right2left[i + 1] + 1

        for i in range(n):
            sum += max(left2right[i], right2left[i])
            
        return sum

### Solution 3, Single Array Optimized Two Pass
Solution description
- Time Complexity: O(N)
  - The array candies of size n is traversed twice left to right and right to left.
- Space Complexity: O(N)
  - An array candies of size n is used.

In [ ]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        candies = [1] * len(ratings)

        for i in range(1, len(ratings)):
            if ratings[i] > ratings[i - 1]:
                candies[i] = candies[i - 1] + 1

        sum = candies[-1]
        for i in range(len(ratings) - 2, -1, -1):
            if ratings[i] > ratings[i + 1]:
                candies[i] = max(candies[i], candies[i + 1] + 1)
            sum += candies[i]
            
        return sum

### Solution 4, Most Optimum Single Pass Approach with Constant Space
Solution description
- Time Complexity: O(N)
- Space Complexity: O(1)

In [ ]:
class Solution:
    # Function to calculate sum of first n natural numbers
    def count(self, n):
        return (n * (n + 1)) // 2

    def candy(self, ratings: List[int]) -> int:
        if len(ratings) <= 1:
            return len(ratings)

        candies = 0
        up = 0
        down = 0
        oldSlope = 0


        """
        Accumulates up while going uphill
        At the peak (uphill → downhill), it switches to accumulating down
        Only finalizes the mountain when we hit flat or a valley (downhill → uphill/flat)

        The reason we defer counting until the end of a mountain sequence is because the peak's candy count depends on BOTH slopes — and we don't know the downhill length until we've traversed it.
        """
        for i in range(1, len(ratings)):
            newSlope = (
                1
                if ratings[i] > ratings[i - 1]
                else (-1 if ratings[i] < ratings[i - 1] else 0)
            )
            # End of a mountain sequence:
            # - Uphill followed by flat, OR
            # - Downhill followed by flat or uphill
            if (oldSlope > 0 and newSlope == 0) or (
                oldSlope < 0 and newSlope >= 0
            ):
                # count(up) - Candies for uphill slope (excluding peak)
                # count(down) - Candies for downhill slope (excluding peak)
                # max(up, down)	- Candies for the peak
                candies += self.count(up) + self.count(down) + max(up, down)
                up = 0
                down = 0
            
            # slope is uphill
            if newSlope > 0:
                up += 1
            
            # slope is downhill
            elif newSlope < 0:
                down += 1
            
            # slope is flat
            else:
                candies += 1
            
            oldSlope = newSlope

        # This line is needed because the for loop only finalizes mountains when a boundary is hit, but the last mountain might extend to the end of the array without hitting any boundary
        # The +1 accounts for the first child in the sequence (the starting point).
        candies += self.count(up) + self.count(down) + max(up, down) + 1
        return candies